# IAU projekt

> Roman Bitarovský, Emma Macháčová

## Table of contents
* [Zadanie](#zadanie)
    * [Slovníček](#slovnicek)
   
1. [Fáza 1](#faza1)
    1. [Základný opis dát spolu s ich charakteristikami](#zakladnyOpis)
        1. [Počet záznamov, počet atribútov, ich typy](#111)
        2. [Distribúcie, základné deskriptívne štatistiky a pod.](#112)
        3. [Párová analýza dát: vzťahy medzi zvolenými dvojicami atribútov, závislostí medzi dvojicami atribútov, závislosti medzi predikovanou premennou a ostatnými premennými](#113)
    2. [Identifikácia problémov v dátach s prvotným riešením](#identifikacia)
        1. [Duplicitné záznamy](#121)
        2. [Nejednotné formáty](#122)
        3. [Chýbajúce hodnoty](#123)
        4. [Nevhodná štruktúra dát a data merge](#124)
    3. [Formulácia a štatistické overenie hypotéz o dátach](#hypotezy)
2. [](#)

## Zadanie <a name="zadanie"></a>
Znečistenie ovzdušia spôsobuje vážne dýchacie a srdcové ochorenia, ktoré môžu byť smrteľné. Najčastejšie sú postihnuté deti, čo vedie k zápalu pľúc a problémom s dýchaním vrátane astmy. Kyslé dažde, ničenie ozónovej vrstvy a globálne otepľovanie sú niektoré z nepriaznivých dôsledkov. Dátová sada pre Vás (World's Air Pollution: Real-time Air Quality Index https://waqi.info/) predstavuje záznamy jednotlivých meraní kvality ovzdušia ako kombinácia mnohých faktorov bez časovej následnosti. V záznamoch je závislá premenná s menom “warning” indikujúca alarmujúci stav kvality ovzdušia. Vo veľkých mestách ako napr. Peking (angl. Beijing, hlavné mesto Číny s viac ako 21 miliónov ľudí) sa pri varovaní spustí opatrenie ako obmedzenie pohybov áut a ľudí v meste alebo umelý dážď až pokiaľ kvalita vzduchu sa nevráti do normu.

* Úlohou je predikovať závislé hodnoty premennej “warning” pomocou metód strojového učenia.
* Pritom sa treba vysporiadať s viacerými problémami, ktoré sa v dátach nachádzajú ako formáty dát, chýbajúce, vychýlené hodnoty a pod.


### Slovníček  <a name="slovnicek"></a>
<details>
    <summary>Zobraziť</summary>
    
    PM2.5 - Particulate Matter (µg/m3) 
    PM10 - Particulate Matter (µg/m3) 
    NOx - Nitrogen Oxides (µg/m3)
    NO2 - Nitrogen Dioxide (µg/m3)
    SO2 - Sulfur Dioxide  (µg/m3)
    CO - Carbon Monoxide emissions  (µg/m3)
    CO2 - Carbon Dioxide  (µg/m3)
    PAHs - Polycyclic Aromatic Hydrocarbons  (µg/m3)
    NH3 - Ammonia trace  (µg/m3)
    Pb - Lead  (µg/m3)
    TEMP - Temperature (degree Celsius)
    DEWP - Dew point temperature (degree Celsius)
    PRES - Pressure (hPa, <100, 1050>)
    RAIN - Rain (mm)
    WSPM - Wind Speed (m/s)
    WD - Wind Direction
    VOC - Volatile Organic Compounds
    CFCs - Chlorofluorocarbons
    C2H3NO5 - Peroxyacetyl nitrate
    H2CO - Plywood emit formaldehyde
    GSTM1 - Glutathione-S transferase M1
    1-OHP - 1-hydroxypyrene
    2-OHF - 2-hydroxyfluorene
    2-OHNa - 2-hydroxynaphthalene
    N2 - Nitrogen
    O2 - Oxygen
    O3 - Ozone
    Ar - Argon
    Ne - Neon
    CH4 - Methane
    He - Helium
    Kr - Krypton
    I2 - Iodine
    H2 - Hydrogen
    Xe - Xenon
</details>

## Data init

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway

In [ ]:
labor_measurements = pd.read_csv('../081/measurements.csv', sep='\t')
labor_stations = pd.read_csv('../081/stations.csv', sep='\t')

## Fáza 1  <a name="faza1"></a>
### 1.1 Základný opis dát spolu s ich charakteristikami (5b) <br>  <a name="zakladnyOpis"></a>
V tejto fáze uveďte:
* Počet záznamov, počet atribútov, ich typy,
* Pre zvolené významné atribúty ich distribúcie, základné deskriptívne štatistiky a pod.
* Párová analýza dát: preskúmajte vzťahy medzi zvolenými dvojicami atribútov. 
* Párová analýza dát: Identifikujte závislostí medzi dvojicami atribútov (napr. korelácie)
* Párová analýza dát: Identifikujte závislosti medzi predikovanou premennou a ostatnými premennými (potenciálnymi prediktormi).

In [ ]:
labor_measurements.head()

#### 1.1.1 Počet záznamov, počet atribútov, ich typy <a name="111"></a>

In [ ]:
print('Počet záznamov: ', labor_measurements.shape[0])
print('Počet atribútov: ', labor_measurements.shape[1])
print('\nTypy atribútov:')
display(labor_measurements.dtypes)

In [ ]:
labor_measurements.describe()

In [ ]:
labor_stations.head()

In [ ]:
print('Počet záznamov: ', labor_stations.shape[0])
print('Počet atribútov: ', labor_stations.shape[1])
print('\nTypy atribútov:')
display(labor_stations.dtypes)

In [ ]:
labor_stations.describe()

### 1.2. Identifikácia problémov v dátach s prvotným riešením (5b)  <a name="identifikacia"></a>
* Identifikujte problémy v dátach napr.: 
    * nevhodná štruktúra dát, 
    * duplicitné záznamy, 
    * nejednotné formáty, 
    * chýbajúce hodnoty, 
    * vychýlené hodnoty,
    * v dátach sa môžu nachádzať aj iné, tu nevymenované problémy. 
* Navrhnuté riešenie problémov s dátami prvotne realizujte na dátach. Problémy s dátami môžete riešiť iteratívne v každej fáze aj vo všetkých fázach podľa Vašej potreby.

#### 1.2.1 Duplicitné záznamy <a name="121"></a>

In [ ]:
if (labor_measurements.duplicated().any()):
    print('labor_measurements pocet duplikatov', len(labor_measurements)-len(labor_measurements.drop_duplicates()))
else:
    print('labor_measurements no duplicates')

In [ ]:
if (labor_stations.duplicated().any()):
    print('labor_stations pocet duplikatov', len(labor_stations)-len(labor_stations.drop_duplicates()))
else:
    print('labor_stations no duplicates')

#### 1.2.2 Nejednotné formáty <a name="122"></a>

In [ ]:
labor_stations.columns.values

In [ ]:
qos_count = labor_stations['QoS'].value_counts()
qos_count

In [ ]:
revision_count = labor_stations['revision'].value_counts()
revision_count

#### 1.2.3 Chýbajúce hodnoty <a name="123"></a>

In [ ]:
labor_measurements.replace('', np.nan, inplace=True)

In [ ]:
labor_stations.replace('', np.nan, inplace=True)

In [ ]:
temp = labor_measurements.shape[0] - labor_measurements.dropna().shape[0]
print('Chýbajúce hodnoty pre labor_measurements: ', temp)

In [ ]:
temp = labor_stations.shape[0] - labor_stations.dropna().shape[0]
print('Chýbajúce hodnoty pre labor_stations: ', temp)

#### 1.2.4. Nevhodná štruktúra dát a data merge <a name="124"></a>
Najskôr dropneme zo labor_stations nepotrebné stĺpce a následne aj duplikáty

In [ ]:
labor_stations.drop(labor_stations.columns[1], axis=1, inplace=True)
labor_stations.drop(labor_stations.columns[2:4], axis=1, inplace=True)
labor_stations.head()

Následne vykonáme merge na základe súradníc

In [ ]:
# Table merge
df = pd.merge(labor_measurements, labor_stations, how='inner', left_on=['latitude', 'longitude'], right_on=['latitude', 'longitude'])

Dropneme stĺpce obahujúce súradnice lebo ich už viac nepotrebujem vzhľadom na stĺpec location

In [ ]:
df = df.drop(columns=['latitude', 'longitude'])

Dropneme neplatné hodnoty

In [ ]:
# df = df.dropna()
# toto zatial nechame lebo sa mohla viac krat namerat tá ista hodnota v inom case
# df.drop_duplicates()

In [ ]:
print('Počet záznamov: ', df.shape[0])
print('Počet atribútov: ', df.shape[1])
print('\nTypy atribútov:')
display(df.dtypes)

Reorganizujeme poradie stĺpcov

In [ ]:
df.columns.values
df = df[['location', 'warning', 'TEMP', 'PRES', 'PM2.5', 'NOx', 'PM10', 'C2H3NO5', 'CH4', 'Pb', 'NH3', 'SO2', 'O3', 'CO', 'PAHs', 'H2CO', 'CFCs']]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
war_count = df['warning'].value_counts()
war_count

In [ ]:
war_count.plot(kind='pie', title='Warning visual ratio')

#### 1.1.2 Distribúcie, základné deskriptívne štatistiky a pod. <a name="112"></a>

In [ ]:
df.loc[:, df.columns.difference(["index"])].hist(sharex=False, sharey=False, figsize=(20, 40), bins=50)

In [ ]:
# jediná nenormálová zaujímavá distribúcia
df['C2H3NO5'].plot(kind='kde')
plt.title('Distribution of O3')
plt.xlabel('Percentage')

In [ ]:
df.loc[:, df.columns.difference(["index"])].plot(kind='box', subplots=True, sharex=False, sharey=False, figsize=(60, 20), layout=(4, 4))

#### 1.1.3 Párová analýza dát <a name="113"></a>

In [ ]:
#sns.pairplot(df, hue="warning")   # toto asi odkomentovat potom

In [ ]:
# analysis of correlations
df_nums_only = df.copy()
df_nums_only = df_nums_only.drop(columns=['location'])
df_nums_only_corr = df_nums_only.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
sns.heatmap(df_nums_only_corr, ax=ax, annot=True, cmap='coolwarm')

In [ ]:
df_nums_only_corr[((df_nums_only_corr >= 0.39) | (df_nums_only_corr <= -0.39)) & (df_nums_only_corr != 1)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(df_nums_only_corr[((df_nums_only_corr >= 0.32) | (df_nums_only_corr <= -0.32)) & (df_nums_only_corr != 1)], ax=ax, annot=True, cmap='coolwarm')

Významnejšie korelácie sa javia byť medzi týmito atribútmi:
* PM2.5 - PM10
* PM10 - NH3
* PM10 - C2H3NO5

In [ ]:
# nič moc korelácia
sns.scatterplot(x=df_nums_only['PM10'], y=df_nums_only['PM2.5'], hue=df_nums_only['warning'])

In [ ]:
# nič moc korelácia
sns.scatterplot(x=df_nums_only['NH3'], y=df_nums_only['PM10'], hue=df_nums_only['warning'])

In [ ]:
# good korelácia
sns.scatterplot(x=df_nums_only['C2H3NO5'], y=df_nums_only['PM10'], hue=df_nums_only['warning'])

In [ ]:
corr1=df['warning'].corr(df['C2H3NO5'])
corr2=df['warning'].corr(df['PM10'])
corr3=df['PM10'].corr(df['C2H3NO5'])
print(corr1, corr2, corr3)

In [ ]:
g = sns.displot(
    data=df, y="TEMP", hue="warning", col="C2H3NO5",
    kind="kde", height=4, aspect=.7,
)
g.set_axis_labels("Density (a.u.)", "Temperature (Celsius)")
g.set_titles("{col_name} df")

### 1.3 Formulácia a štatistické overenie hypotéz o dátach (5b) <br> <a name="hypotezy"></a>
* Sformulujte dve hypotézy o dátach v kontexte zadanej predikčnej úlohy. 
* Príklad formulovania hypotézy: merania kvality ovzdušia v kritickom stave majú v priemere inú (vyššiu/nižšiu) hodnotu určitej chemikálie (alebo koncentrácie látok) ako merania kvality ovzdušia v normálnom stave.
* Sformulované hypotézy overte vhodne zvoleným štatistickým testom.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

> #### Hypotéza 1: Merania kvality ovzdušia majú v kritickom stave v priemere vyššiu hodnotu látky C2H3NO5
> H0 - merania kvality ovzdušia majú v kritickom stave v priemere vyššiu hodnotu látky C2H3NO5 <br>
> H1 - merania kvality ovzdušia nemajú v kritickom stave v priemere vyššiu hodnotu látky C2H3NO5

In [ ]:
gas_warning = df[df["warning"] == 1.0]
gas_no_warning = df[df["warning"] == 0]     

In [ ]:
print ("vyskyt nan vo vyberoch:", gas_warning['C2H3NO5'].isnull().values.any(), gas_no_warning['C2H3NO5'].isnull().values.any())

In [ ]:
gas_warning = gas_warning[df.C2H3NO5.notnull()]
gas_no_warning = gas_no_warning[df.C2H3NO5.notnull()]

In [ ]:
print ("vyskyt nan vo vyberoch:", gas_warning['C2H3NO5'].isnull().values.any(), gas_no_warning['C2H3NO5'].isnull().values.any())

In [ ]:
gas_warning_list = gas_warning['C2H3NO5'].tolist()
gas_no_warning_list = gas_no_warning['C2H3NO5'].tolist()

##### Test normality

In [ ]:
st1 = stats.shapiro(gas_warning_list)
st2 = stats.shapiro(gas_no_warning_list)

print(st1, st2)

Mann Whitney U test

In [ ]:
#mannwhitneyu(gas_warning_list, gas_no_warning_list, method="exact")   # toto treba nejak zredukovat data lebo to nezbehne

##### Kruskalov test

In [ ]:
stats.kruskal(gas_warning_list, gas_no_warning_list)   # neviem ci na toto nemusia byt z normalneho

##### Analysis of variance test

In [ ]:
f_oneway(gas_warning_list, gas_no_warning_list)    # neviem ci na toto nemusia byt z normalneho

##### Plots

In [ ]:
sns.distplot(gas_warning_list)

In [ ]:
sns.distplot(gas_no_warning_list)

In [ ]:
df.boxplot(by ='warning', column =['C2H3NO5'], grid = True)

> Na základe testu vyvraciame alternatívnu hypotézu v prospech nulovej hypotézy, a teda nulovú hypotézu akceptujeme. # asi